In [1]:
import numpy as np
import pandas as pd
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import keras

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K 
from tensorflow.python.keras.layers import Layer

from utils.attention import AttentionLayer

from keras.models import load_model
from keras.utils import plot_model

C:\Users\hp\AppData\Local\Temp\ipykernel_216580\1500256061.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
attn_layer = AttentionLayer(name = 'attention_layer')

In [2]:
dataset = pd.read_csv(r"D:\Datasets\news_summary_more.csv")

In [3]:
dataset.head()

NameError: name 'dataset' is not defined

In [6]:
dataset.drop_duplicates(inplace=True)

Preprocessing Function

In [7]:
# def preprocess(text):
#     text = text.lower()  # lowering the text
#     text = [contractions.fix(word) for word in text.split(" ")]  # word contraction
#     text = " ".join(text)
#     text = [word for word in text.split(" ") if word not in stopwords.words('english')]
#     text = " ".join(text)
#     text = re.sub(r"\'s ","",text)   # remove 's from the sentence
#     text =  re.sub(r"\(.*\)","",text) # remove words written in ()
#     text = re.sub(r'[^a-zA-Z0-9. ]','',text)
#     text = re.sub(r'\.','. ',text)
#     text = re.sub(r'\s+',' ',text)
#     return text

In [9]:
# applying preprocessing function on text and headlines

# dataset['headlines'] = dataset['headlines'].apply(preprocess)
# dataset['text'] = dataset['text'].apply(preprocess)

KeyboardInterrupt: 

In [10]:
# dataset.to_csv("dataset_new.csv")

loading the processed dataset

In [4]:
dataset_new = pd.read_csv("dataset_new.csv")

In [5]:
dataset_new.head()

,Unnamed: 0,headlines,text
0,0,upgrad learner switches career ml al 90 salary...,saurav kant alumnus upgrad iiitbpg program mac...
1,1,delhi techie wins free food swiggy one year cred,kunal shahcredit card bill payment platform cr...
2,2,new zealand end rohit sharmaled india12match w...,new zealand defeated india 8 wickets fourth od...
3,3,aegon life iterm insurance plan helps customer...,aegon life iterm insurance plan customers enjo...
4,4,known hirani yrs metoo claims true sonam,speaking sexual harassment allegations rajkuma...


In [6]:
dataset_new['headlines'] = dataset_new["headlines"].apply(lambda x: "_start_ " + x +" _end_")

In [7]:
input_lengths = []
output_lengths = []

for i,j in zip(dataset_new["headlines"],dataset_new['text']):
    input_lengths.append(len(j.split()))
    output_lengths.append(len(i.split()))

word embedding

In [8]:
embedding_matrix = {}
with open(r"D:\Datasets\glove\glove.42B.300d.txt\glove.42B.300d.txt",encoding="utf8") as file:
    for line in file:
        values =  line.split()
        word = values[0]
        vector = np.array(values[1:],'float32')
        embedding_matrix[word] = vector

In [9]:
# counting unique words from input text since the output is the summary the words would be present in it.

word_corpus = set()

for line in dataset_new['text']:
    for word in line.split():
        if word not in word_corpus:
            word_corpus.add(word)
            
unique_words = len(word_corpus)

# words present in unique words and corpus

# inter_words = []

# for word in embedding_matrix.keys():
#     if word in word_corpus:
#         inter_words.append(word)

inter_words = set(embedding_matrix.keys()).intersection(word_corpus)

print("total words present {}".format(len(word_corpus)))
print('total interleaving words present are {} %'.format(np.round((float(len(inter_words))/len(word_corpus))*100)))

total words present 161005
total interleaving words present are 46.0 %


In [10]:
words_source_train = []
for i in dataset_new['text'] :
  words_source_train.extend(i.split(' '))

print("all the words in the corpus", len(words_source_train))
words_source_train = set(words_source_train)
print("the unique words in the corpus", len(words_source_train))
inter_words = set(embedding_matrix.keys()).intersection(words_source_train)
print("The number of words that are present in both glove vectors and our corpus are {} which \
is nearly {}% ".format(len(inter_words), np.round((float(len(inter_words))/len(words_source_train))
*100)))

all the words in the corpus 3618225
the unique words in the corpus 161006
The number of words that are present in both glove vectors and our corpus are 73849 which is nearly 46.0% 


In [11]:
word_source_corpus = {}

glove = set(embedding_matrix.keys())

for word in word_corpus:
    if word in glove:
        word_source_corpus[word] = embedding_matrix[word]
        
len(word_source_corpus)

73849

In [12]:
def num(text):
    unique_w = [w for w in text.split() if w not in inter_words]
    return len(unique_w)

dataset_new['unique_words'] = dataset_new['text'].apply(num)

In [13]:
dataset_new.head()

,Unnamed: 0,headlines,text,unique_words
0,0,_start_ upgrad learner switches career ml al 9...,saurav kant alumnus upgrad iiitbpg program mac...,5
1,1,_start_ delhi techie wins free food swiggy one...,kunal shahcredit card bill payment platform cr...,4
2,2,_start_ new zealand end rohit sharmaled india1...,new zealand defeated india 8 wickets fourth od...,3
3,3,_start_ aegon life iterm insurance plan helps ...,aegon life iterm insurance plan customers enjo...,0
4,4,_start_ known hirani yrs metoo claims true son...,speaking sexual harassment allegations rajkuma...,2


In [14]:
dataset_new = dataset_new[dataset_new['unique_words'] < 4]

dataset_new.reset_index(inplace=True,drop=True)

dataset_new.head()

,Unnamed: 0,headlines,text,unique_words
0,2,_start_ new zealand end rohit sharmaled india1...,new zealand defeated india 8 wickets fourth od...,3
1,3,_start_ aegon life iterm insurance plan helps ...,aegon life iterm insurance plan customers enjo...,0
2,4,_start_ known hirani yrs metoo claims true son...,speaking sexual harassment allegations rajkuma...,2
3,5,_start_ rahat fateh ali khan denies getting no...,pakistani singer rahat fateh ali khan denied r...,2
4,7,_start_ govt directs alok verma join work 1 da...,weeks excbi director alok verma told departmen...,1


splitting the data into training and testing set

In [15]:
X_train,X_val,y_train,y_val = train_test_split(dataset_new['text'],dataset_new['headlines'],test_size=5,random_state=0)
X_test, X_val, y_test, y_val = train_test_split(X_val, y_val, test_size = 0.5, random_state = 20)

In [16]:
max_text_len = max(input_lengths)
max_headline_len = max(output_lengths)

print(max_text_len)
print(max_headline_len)

66
16


Tokenization

In [17]:
x_t = Tokenizer()

x_t.fit_on_texts(dataset_new['text'] + dataset_new['headlines'])

x_vocab_size = len(x_t.word_index) + 1

encoded_x_train = x_t.texts_to_sequences(X_train)
encoded_x_val = x_t.texts_to_sequences(X_val)
encoded_x_test = x_t.texts_to_sequences(X_test)

padded_X_train = pad_sequences(encoded_x_train, maxlen=max_text_len,padding='post')
paddad_X_val = pad_sequences(encoded_x_val,maxlen=max_text_len,padding='post')
padded_X_test = pad_sequences(encoded_x_test,maxlen=max_text_len,padding='post')



y_t = Tokenizer()
y_t.fit_on_texts(dataset_new['headlines'])
y_vocab_size = len(y_t.word_index) + 1

encoded_y_train = y_t.texts_to_sequences(y_train)
encoded_y_val = y_t.texts_to_sequences(y_val)
encoded_y_test = y_t.texts_to_sequences(y_test)

paddad_y_train = pad_sequences(encoded_y_train,maxlen=max_headline_len,padding='post')
paddad_y_test = pad_sequences(encoded_y_test,maxlen=max_headline_len,padding='post')
padded_y_val = pad_sequences(encoded_y_val,maxlen=max_headline_len,padding='post')

In [18]:
# Embedding matrix
embedding_dim = 300
embedding_matrix_n = np.zeros((x_vocab_size,embedding_dim),dtype=np.float32)

for word,i in x_t.word_index.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_matrix_m = np.zeros((x_vocab_size,embedding_dim),dtype=np.float32)

for word,i in x_t.word_index.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        embedding_matrix_m[i] = embedding_vector

Encoder Decoder Neural Network

In [26]:
encoder_input_layer = Input(shape=(max_text_len,), name="encoder_input_layer")

# The Embedding Layer

embedding_layer_input = Embedding(input_dim=x_vocab_size, output_dim=300, trainable=False, input_length=max_text_len, weights=[embedding_matrix], name="encoder_embedding_layer")(encoder_input_layer)

# print(embedding_layer_input.shape)

# encoder

encoder_lstm_1,state_h1,state_c1 = LSTM(500, return_sequences= True,return_state=True, name="encoder_lstm_1")(embedding_layer_input)
encoder_lstm_2,state_h2,state_c2 = LSTM(500, return_sequences= True,return_state=True, name="encoder_lstm_2")(encoder_lstm_1)
encoder_lstm_3,state_h3,state_c3 = LSTM(500, return_sequences= True,return_state=True, name="encoder_lstm_3")(encoder_lstm_2)

output_encoder, state_h, state_c = LSTM(500, return_state=True, return_sequences=True, name="encoder_output_layer")(encoder_lstm_3)
encoder_states = [state_h,state_c]


decoder_input_layer = Input(shape=(None,), name="decoder_input_layer")

embedding_layer_output = Embedding(input_dim=y_vocab_size, output_dim=300, trainable=False, input_length=max_text_len, weights=[embedding_matrix], name="decoder_embedding_layer")(decoder_input_layer)

# decoder

# decoder_lstm_1,_,_ = LSTM(256, return_sequences=True, name="decoder_lstm_1")(embedding_layer_output,initial_state=encoder_states)

output_decoder, state_h, state_c = LSTM(500, return_state=True, return_sequences=True, name="decoder_output_layer")(embedding_layer_output,initial_state=encoder_states)

## Attention Layer
attn_layer = AttentionLayer(name = 'attention_layer')
attn_out, attn_states = attn_layer([output_encoder, output_decoder])   # based on the encoder and decoder outputs, determine which nodes are more important

# ## Concat attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([output_decoder, attn_out])

## Dense layer
decoder_dense = TimeDistributed(Dense(y_vocab_size, activation='softmax'))(decoder_concat_input)

# decoder_states = [state_h,state_c]
# output_dense = Dense(len(tokens_output), activation='softmax',name="output_dense")(output_decoder)

model = Model(inputs=[encoder_input_layer,decoder_input_layer], outputs=[decoder_dense])

In [27]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
checkpoint_filepath = './model.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True, save_freq = "epoch")
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1)
history=model.fit([padded_X_train,paddad_y_train[:,:-1]], paddad_y_train.reshape(paddad_y_train.shape[0],paddad_y_train.shape[1], 1)[:,1:] ,epochs=10,batch_size=512, validation_data=([paddad_X_val,padded_y_val[:,:-1]], padded_y_val.reshape(padded_y_val.shape[0],padded_y_val.shape[1], 1)[:,1:]))

Epoch 1/10
  2/149 [..............................] - ETA: 1:05:23 - loss: 10.6062

KeyboardInterrupt: 

In [66]:
custom_objects = {'AttentionLayer': AttentionLayer}
with keras.utils.custom_object_scope(custom_objects):
    loaded_model = load_model(r"C:\Users\hp\Downloads\test_model.h5")
    # loaded_model = load_model(r"D:\vs code\python\DeepLearning\Projects\textSummarizer\Abstractive\utils\summariser_2.h5")

In [67]:
loaded_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input_layer (Input  [(None, 66)]                 0         []                            
 Layer)                                                                                           
                                                                                                  
 encoder_embedding_layer (E  (None, 66, 300)              3320460   ['encoder_input_layer[0][0]'] 
 mbedding)                                                0                                       
                                                                                                  
 encoder_lstm_1 (LSTM)       [(None, 66, 500),            1602000   ['encoder_embedding_layer[0][0
                              (None, 500),                          ]']                     

In [68]:
plot_model(loaded_model,show_shapes=True,show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [69]:
for i, layer in enumerate(loaded_model.layers):
    print(f"Layer at index {i}: {layer.name}, Type: {type(layer)}")

Layer at index 0: encoder_input_layer, Type: <class 'keras.src.engine.input_layer.InputLayer'>
Layer at index 1: encoder_embedding_layer, Type: <class 'keras.src.layers.core.embedding.Embedding'>
Layer at index 2: encoder_lstm_1, Type: <class 'keras.src.layers.rnn.lstm.LSTM'>
Layer at index 3: encoder_lstm_2, Type: <class 'keras.src.layers.rnn.lstm.LSTM'>
Layer at index 4: decoder_input_layer, Type: <class 'keras.src.engine.input_layer.InputLayer'>
Layer at index 5: encoder_lstm_3, Type: <class 'keras.src.layers.rnn.lstm.LSTM'>
Layer at index 6: decoder_embedding_layer, Type: <class 'keras.src.layers.core.embedding.Embedding'>
Layer at index 7: encoder_output_layer, Type: <class 'keras.src.layers.rnn.lstm.LSTM'>
Layer at index 8: decoder_output_layer, Type: <class 'keras.src.layers.rnn.lstm.LSTM'>
Layer at index 9: attention_layer, Type: <class 'utils.attention.AttentionLayer'>
Layer at index 10: concat_layer, Type: <class 'keras.src.layers.merging.concatenate.Concatenate'>
Layer at in

In [70]:
loaded_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input_layer (Input  [(None, 66)]                 0         []                            
 Layer)                                                                                           
                                                                                                  
 encoder_embedding_layer (E  (None, 66, 300)              3320460   ['encoder_input_layer[0][0]'] 
 mbedding)                                                0                                       
                                                                                                  
 encoder_lstm_1 (LSTM)       [(None, 66, 500),            1602000   ['encoder_embedding_layer[0][0
                              (None, 500),                          ]']                     

Developing inference model

In [71]:
input_encoder = loaded_model.layers[0].input   # taking input

embedding_inp = loaded_model.layers[1](input_encoder)

# Encoder
lstm_enc_1,h1,c1 = loaded_model.layers[2](embedding_inp) # encoder lstm 1
lstm_enc_2,h2,c2 = loaded_model.layers[3](lstm_enc_1) # encoder lstm 2
lstm_enc_3,h3,c3 = loaded_model.layers[5](lstm_enc_2) # encoder lstm 3
lstm_enc_output,h,c = loaded_model.layers[7](lstm_enc_3) # encoder output layer
encoder_states = [h,c]

model_enc = Model(input_encoder,[lstm_enc_output,encoder_states])  # final encoder model

decoder_initial_state_h = Input(shape=(500,))
decoder_initial_state_c = Input(shape=(500,))

encoder_out = Input(shape=(max_text_len,500,))

decoder_states = [decoder_initial_state_h,decoder_initial_state_c]

input_decoder = loaded_model.layers[4].input # taking input

embedding_out = loaded_model.layers[6](input_decoder)

# Decoder
lstm_dec_output,h_d,c_d = loaded_model.layers[8](embedding_out,initial_state=decoder_states) # decoder output layer

attention_output,attention_states = loaded_model.layers[9]([encoder_out,lstm_dec_output]) # implementing attention mechanism

concatenation_output = loaded_model.layers[10]([lstm_dec_output,attention_output]) # concatenation layer

time_distributed_layer = loaded_model.layers[11](concatenation_output) # time distributed layer that provides the predicted word

decoder_model = Model([input_decoder] + [encoder_out,decoder_states], [time_distributed_layer] + [h_d,c_d]) # final decoder model

In [72]:
reverse_target_word = y_t.index_word
reverse_input_word = x_t.index_word
target_word_index = y_t.word_index

In [73]:
reverse_target_word

{1: 'end',
 2: 'start',
 3: 'us',
 4: 'india',
 5: 'man',
 6: 'govt',
 7: 'pm',
 8: 'delhi',
 9: '2',
 10: 'trump',
 11: 'report',
 12: 'bjp',
 13: 'indian',
 14: 'cm',
 15: 'woman',
 16: '1',
 17: 'new',
 18: 'modi',
 19: '3',
 20: 'crore',
 21: '1st',
 22: 'people',
 23: 'police',
 24: 'mumbai',
 25: 'years',
 26: 'sc',
 27: 'reports',
 28: 'get',
 29: 'killed',
 30: '5',
 31: 'first',
 32: 'lakh',
 33: 'world',
 34: 'case',
 35: 'time',
 36: 'video',
 37: 'pak',
 38: 'film',
 39: 'china',
 40: 'gets',
 41: 'minister',
 42: 'says',
 43: 'rahul',
 44: 'arrested',
 45: '4',
 46: '10',
 47: 'may',
 48: 'chief',
 49: 'women',
 50: 'ceo',
 51: 'found',
 52: 'day',
 53: 'uk',
 54: 'google',
 55: 'test',
 56: 'mp',
 57: 'win',
 58: 'congress',
 59: 'like',
 60: 'cr',
 61: 'hc',
 62: '6',
 63: 'make',
 64: 'made',
 65: 'air',
 66: 'kohli',
 67: 'team',
 68: 'held',
 69: 'attack',
 70: 'asks',
 71: 'court',
 72: 'prez',
 73: 'study',
 74: 'pakistan',
 75: 'death',
 76: 'yrs',
 77: 'shares',
 

In [74]:
def summarise(sentence):
    sentence= sentence.reshape(1,max_text_len)
    encoder_out,encoder_states = model_enc(sentence)
    
    target_sentence = ''
    target_seq = np.zeros((1,1))
    target_seq[0,0] = target_word_index['start']
    stop_condition = False
    
    enc_out,states_enc = model_enc.predict(sentence)
    
    decoder_stat = states_enc
    
    while not stop_condition:
        
        word,s_h,s_c = decoder_model.predict([target_seq] + [enc_out,decoder_stat])
        
        token = np.argmax(word[0,-1,:])
        
        target_word = reverse_target_word[token]
        
        if token != 1:
            target_sentence = target_sentence + target_word + ' '
            
        if token == 1 or len(target_sentence.split()) >= (max_headline_len) - 1:
            stop_condition = True
            
        target_seq = np.zeros((1,1))
        target_seq[0,0] = token
        decoder_stat = [s_h,s_c]
        
    return target_sentence

In [75]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
        newString=newString+reverse_target_word[i]+' '
    return newString

In [76]:
def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_input_word[i]+' '
    return newString

In [77]:
for i in range(10):
  print("Review:",seq2text(padded_X_test[i]))
  print("Original summary:",seq2summary(paddad_y_test[i]))
  print("Predicted summary:",summarise(padded_X_test[i]))
  print("\n")

Review: iranislamic revolutionary guard corps sunday declared victory antigovernment protests blamed enemies irgc blamed countries including us israel saudi arabia unrest many 22 people killed 1000 arrested protests began december according iranian officials 
Original summary: iran declares victory antigovernment protests 
1/1 [==============================] - 0s 25ms/step
Predicted summary: isis leaderdeath us us jerusalem iraq 


Review: serving 14 months ceo paytm payments bank renu satti resigned serve coo paytmnew initiative new retail paytm reportedly talks usbased investors existing shareholder softbank raise 500 million same new retail coined jack ma founder alibaba also backs paytm 
Original summary: paytm payments bank ceo resigns lead new retail 
1/1 [==============================] - 0s 34ms/step
Predicted summary: flipkart appoints mohit agarwal quits facebook 




IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
padded_X_test

array([[22948,  8600,  2073,  8247,   108,   706,  1216,  8216,   747,
         1842,  7763, 28893,  1842,   332,    97,     5,  1267,   478,
         1374,  6553,   210,   781,     8,    70,   820,    62,   747,
         1152,   408,    66,  1804,    84,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [ 1636,   543,   171,   128,  1739,  1115,   121, 18490, 24438,
         1467,  1881,  4262, 69460,  1778,    20,  2328,  1739,    28,
          708,  1049,  1275,  2008,  5815,  2298,  1235,   725,    93,
         2357,    20,  2328, 11910,  2331, 14418,   932,  3558,     7,
         6371,  1739,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     

In [ ]:
def BLEU_Score(y_test, y_pred):
    references = [[seq2summary(y_test).split(" ")]]
    candidates = [summarise(y_pred.reshape(1, max_text_len)).split(" ")]

    # Apply smoothing function
    smooth = SmoothingFunction().method4
    bleu_score = corpus_bleu(references, candidates, smoothing_function=smooth)
    
    return bleu_score
from nltk.translate.bleu_score import corpus_bleu
scores=[]
for i in range(0,2):
    scores.append(BLEU_Score(paddad_y_test[i],padded_X_test[i]))
    
print(np.mean(scores))

1/1 [==============================] - 0s 72ms/step
0.05025188014078477
